In [1]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import math

In [29]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [ ]:
link='https://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/ophtalmologiste'

In [ ]:
driver.get(link)

In [ ]:
urlstoscrap=[]
soup=BeautifulSoup(driver.page_source,'html.parser')
allplaces=soup.find_all('li',class_='seo-departement')
for a in allplaces:
    href=a.find('a')['href']
    urlstoscrap.append('https://annuairesante.ameli.fr'+href)

In [ ]:
secondariurls=[]
for e in urlstoscrap:
    driver.get(e)
    time.sleep(2)
    soup2=BeautifulSoup(driver.page_source,'html.parser')
    try:
      first=soup2.find('ul',class_='first')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')
    try:
      first=soup2.find('ul',class_='second')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')
    try:
      first=soup2.find('ul',class_='third')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')

In [ ]:

urlstoscrap3=[]

In [ ]:
for index,enlace in enumerate(secondariurls):
    numde=0
    driver.get('https://annuairesante.ameli.fr'+enlace)
    time.sleep(1)
    while numde==0:
      slider = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="site_inner"]/div[7]'))
                )
      soup2=BeautifulSoup(driver.page_source,'html.parser')
      targets=soup2.find_all('div',class_='item-professionnel')
      print(index)
      for target in targets:
         try:
            prename=target.find('h2').text
            preurl=target.find('a')['href']
            if preurl:
               urlstoscrap3.append('https://annuairesante.ameli.fr'+preurl)
         except AttributeError:
            print('there is no element')
      else:
        time.sleep(1)
        next=WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//img[@alt="Page suivante"]'))
        )
      #   next=driver.find_element(By.XPATH, '//img[@alt="Page suivante"]')
        next.click()
      

In [27]:
nombre=[]
numero=[]
especialidad=[]
direccion=[]    
servicios=[]

In [30]:
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

# Función para reiniciar el navegador
def iniciar_driver():
    
    

    return webdriver.Chrome(options=options)

# Inicia el driver por primera vez


# Aquí comienza el scraping
for indice, enlace in enumerate(urlstoscrap3['0'][3000:]):
    driver.get(enlace)
    print(indice + 3000)
    print(enlace)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="centresite"]/div[1]/a[1]'))
        )
        time.sleep(3)
    except TimeoutException:
        pass

    page_source = driver.page_source

    # ✅ Detección de error HTTP 500 (o página caída)
    if ("HTTP ERROR 500" in page_source or
        "ne peut pas traiter cette demande" in page_source or
        "Cette page ne fonctionne pas" in page_source):

        print(f"[!] Error 500 detectado en: {enlace} — Reiniciando navegador")

        # Rellenar con None
        nombre.append(None)
        especialidad.append(None)
        numero.append(None)
        direccion.append(None)
        servicios.append(None)

        # Reiniciar driver
        driver.quit()
        time.sleep(2)
        driver = iniciar_driver()

        continue  # Saltar al siguiente enlace

    # Si la página está bien, hacer scraping normal
    page = BeautifulSoup(page_source, 'html.parser')

    try:
        pre_nombre = page.find('h1').text
        nombre.append(pre_nombre if pre_nombre else None)
    except AttributeError:
        nombre.append(None)

    try:
        pre_especialidad = page.find('h2', class_='item left specialite').text
        especialidad.append(pre_especialidad if pre_especialidad else None)
    except AttributeError:
        especialidad.append(None)

    try:
        pre_numero = page.find('h2', class_='item left tel').text.replace('\xa0', '')
        numero.append(pre_numero if pre_numero else None)
    except AttributeError:
        numero.append(None)

    try:
        pre_direccion = page.find('h2', class_='item left adresse').text.replace('\n', ' ').replace('\t', '').strip()
        direccion.append(pre_direccion if pre_direccion else None)
    except AttributeError:
        direccion.append(None)

    try:
        pre_servicios = page.find('div', id='popinAutocompleteContent').text.replace('\n', ' ').replace('\t', '').strip()
        servicios.append(pre_servicios if pre_servicios else None)
    except AttributeError:
        servicios.append(None)


3000
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTs3OTK7.html
3001
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTo4NTu7.html
3002
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1mjMwOTWz.html
3003
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTM1ODq0.html
3004
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1mjMwOTCw.html
3005
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1mjMwOTGy.html
3006
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTAyNjGy.html
3007
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTo4NDaz.html
3008
https://annuairesante.ameli.fr/professionnels-de-sante/recherche/fiche-detaillee-A7c1lTo4Mzqx.html
3009
https://annuairesante.ameli.fr/professionnels-de-sante/rech

In [31]:
sample2=({
    'nombre':nombre,
    'numero':numero,
    'especialidad':especialidad,
    'direccion':direccion,
    'servicios':servicios   
})

In [32]:
sample2=pd.DataFrame(sample2)

In [33]:
sample2.drop_duplicates()

,nombre,numero,especialidad,direccion,servicios
0,None,None,None,None,None
31,M. ZANLONGHI XAVIER,None,Médecin ophtalmologiste,WEST OPHTA 10 RUE DE LA MAISON NEUVE 35400 S...,None
32,M. ARCADE PIERRE EMMANUEL,None,Médecin ophtalmologiste,WEST OPHTA 10 RUE DE LA MAISON NEUVE 35400 S...,None
33,M. VIANA MICHEL,None,Médecin ophtalmologiste,WEST OPHTA 10 RUE DE LA MAISON NEUVE 35400 S...,None
34,M. BACHET VALENTIN,None,Médecin ophtalmologiste,CLINIQUE DE LA COTE D EMERAUDE CLINIQUE COTE ...,None
...,...,...,...,...,...
6635,M. BAUDU PIERRE,None,Médecin ophtalmologiste,SELARL GROUPE MEDICAL COLIR 2 RUE DU MAINE 9...,None
6636,M. BOULET FLORENT,None,Médecin ophtalmologiste,SELARL GROUPE MEDICAL COLIR 2 RUE DU MAINE 9...,E Examen de la vision I Imagerie de l'œil (é...
6637,M. RAHMANI FRANCOIS,None,Médecin ophtalmologiste,SELARL GROUPE MEDICAL COLIR 2 RUE DU MAINE 9...,C Consultations E Examen de la vision I Im...
6638,Mme PERNOT ANNE,None,Médecin ophtalmologiste,SELARL GROUPE MEDICAL COLIR 2 RUE DU MAINE 9...,C Consultations E Examen de la vision I Im...


In [34]:
sample2.drop_duplicates(inplace=True)
sample2.to_csv('annuairesante_with_services2.csv',index=False)
sample2.to_excel('annuairesante_with_services2.xlsx',index=False)